In [15]:
import pandas as pd
import numpy as np

# Read Input Data

In [16]:
# read an excel file into a pandas dataframe
df_gitcoin = pd.read_csv('./input/vote_coefficients_input.csv')
df_gitcoin.head()

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,status,last_score_timestamp,type,success,rawScore,threshold
0,0x24a5bbf100c33c84d6adf041b80650a27e0f8a9739d6...,0x64a30a4b92a54a2ad5b7cb48106104d3eb1ca743dd09...,19,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9ba961989dd6609ed091f512be947118c40f2291,0xA26d6AEB775a9528c012B748ACe66E3B0FD574cE,5.000000e+15,9.184332,1,DONE,2023-04-25T13:48:59.888771+00:00,ThresholdScoreCheck,True,28.57,15
1,0x3dce13bb4ed8831dfe3382d745ed593a70a5830dcc3c...,0xc401c980df990c4153febcf89b1b78428be1e1f5020d...,6,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9390fa8656a161442928b442300358d82bec28b0,0x9390fA8656A161442928b442300358D82bEC28b0,2.200000e+15,4.094567,1,DONE,2023-04-25T16:55:55.447871+00:00,ThresholdScoreCheck,True,27.21,15
2,0x4cf20243021a45a1a493358b75ef147e264aee102ad0...,0x97589cd10d5be47bef0513d22cbd9ecb9ce06ec6f9ba...,7,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x5136cdfc4d2b1a74774f5137095f82f88af5ec99,0x0035cC37599241D007D0AbA1Fb931C5FA757f7A1,4.000000e+16,74.446665,1,DONE,2023-04-25T17:25:19.667155+00:00,ThresholdScoreCheck,True,28.57,15
3,0x2b032f10cb65d0b9efb758931efb30571f83b076e573...,0xec0268451dd3c3ef3b527ef6478f0e89eb4741103ce7...,16,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x45b79C6bbE15363E87C948823cca846f1baDB280,3.000000e+15,5.583500,1,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15
4,0x0842753b06ff41ad8507a64154262ca08edd7ad6f551...,0xa9bdf73843d882ce496bae1a489c905e26339d987c08...,29,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x5041A1C1DCC760337e99B03DB60FeAF5f6f6c802,3.000000e+15,5.583500,1,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15


In [17]:
# get table of valid tec holders
# extracted from https://dune.com/queries/2457553/4040451
df_tec = pd.read_csv('./input/tec_holders.csv')[['address','tec_tokens_flag']]
df_tec.head()

,address,tec_tokens_flag
0,0x38dfd788db4cfab7b9da57b06cfc06c6e0f33c3b,1
1,0x5b7575494b1e28974efe6ea71ec569b34958f72e,1
2,0x839395e20bbb182fa440d08f850e6c7a8f6f0780,1
3,0xdf290293c4a4d6ebe38fd7085d7721041f927e0a,1
4,0x45602bfba960277bf917c1b2007d1f03d7bd29e4,1


In [18]:
# get table of te academy token holders
# extracted from https://dune.com/queries/2457581
df_tea_dune = pd.read_csv('./input/tea_holders_dune.csv')[['wallet','tea_flag']]
df_tea_tea = pd.read_excel('./input/tea_holders_tea.xlsx')[['wallet','tea_flag']]

df_tea = pd.concat([df_tea_dune, df_tea_tea]).drop_duplicates(subset=['wallet'])

df_tea.head()

,wallet,tea_flag
0,0x68f6f2db8d5d28fb7d837675781c2d391f5f02c0,1
1,0x3e0cf03f718520f30300266dcf4db50ba12d3331,1
2,0x1d1874f9bcea4400a6a3ed85353bf19a764534e5,1
3,0x4daa278b8f90b86fb70859051911003790f4de57,1
4,0xc710f3da5ea5ae7b060166942185458c60c0434f,1


# Calculate Coefficients

In [19]:
# left join the three tables
df_merge = df_gitcoin\
    .merge(df_tec, left_on='voter', right_on='address',how='left')\
    .merge(df_tea, left_on='voter', right_on='wallet',how='left')
df_merge.head()

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,status,last_score_timestamp,type,success,rawScore,threshold,address,tec_tokens_flag,wallet,tea_flag
0,0x24a5bbf100c33c84d6adf041b80650a27e0f8a9739d6...,0x64a30a4b92a54a2ad5b7cb48106104d3eb1ca743dd09...,19,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9ba961989dd6609ed091f512be947118c40f2291,0xA26d6AEB775a9528c012B748ACe66E3B0FD574cE,5.000000e+15,9.184332,1,DONE,2023-04-25T13:48:59.888771+00:00,ThresholdScoreCheck,True,28.57,15,NaN,NaN,0x9ba961989dd6609ed091f512be947118c40f2291,1.0
1,0x3dce13bb4ed8831dfe3382d745ed593a70a5830dcc3c...,0xc401c980df990c4153febcf89b1b78428be1e1f5020d...,6,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9390fa8656a161442928b442300358d82bec28b0,0x9390fA8656A161442928b442300358D82bEC28b0,2.200000e+15,4.094567,1,DONE,2023-04-25T16:55:55.447871+00:00,ThresholdScoreCheck,True,27.21,15,NaN,NaN,NaN,NaN
2,0x4cf20243021a45a1a493358b75ef147e264aee102ad0...,0x97589cd10d5be47bef0513d22cbd9ecb9ce06ec6f9ba...,7,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x5136cdfc4d2b1a74774f5137095f82f88af5ec99,0x0035cC37599241D007D0AbA1Fb931C5FA757f7A1,4.000000e+16,74.446665,1,DONE,2023-04-25T17:25:19.667155+00:00,ThresholdScoreCheck,True,28.57,15,NaN,NaN,NaN,NaN
3,0x2b032f10cb65d0b9efb758931efb30571f83b076e573...,0xec0268451dd3c3ef3b527ef6478f0e89eb4741103ce7...,16,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x45b79C6bbE15363E87C948823cca846f1baDB280,3.000000e+15,5.583500,1,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15,NaN,NaN,NaN,NaN
4,0x0842753b06ff41ad8507a64154262ca08edd7ad6f551...,0xa9bdf73843d882ce496bae1a489c905e26339d987c08...,29,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x5041A1C1DCC760337e99B03DB60FeAF5f6f6c802,3.000000e+15,5.583500,1,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15,NaN,NaN,NaN,NaN


In [20]:
# QF? any other multiplier etc. aggregate to payout by projectid

# multiply coefficient by 1.5 if tec_flag or tea_flag = 1
df_merge['coefficient'] = np.where(df_merge['tea_flag'] == 1,\
                                    df_merge['coefficient'] * 1.5,\
                                    df_merge['coefficient'])
                                    
df_merge['coefficient'] = np.where((df_merge['tec_tokens_flag'] == 1) & (df_merge['coefficient'] == 1), df_merge['coefficient'] * 1.5, df_merge['coefficient'])

df_merge.head()

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,status,last_score_timestamp,type,success,rawScore,threshold,address,tec_tokens_flag,wallet,tea_flag
0,0x24a5bbf100c33c84d6adf041b80650a27e0f8a9739d6...,0x64a30a4b92a54a2ad5b7cb48106104d3eb1ca743dd09...,19,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9ba961989dd6609ed091f512be947118c40f2291,0xA26d6AEB775a9528c012B748ACe66E3B0FD574cE,5.000000e+15,9.184332,1.5,DONE,2023-04-25T13:48:59.888771+00:00,ThresholdScoreCheck,True,28.57,15,NaN,NaN,0x9ba961989dd6609ed091f512be947118c40f2291,1.0
1,0x3dce13bb4ed8831dfe3382d745ed593a70a5830dcc3c...,0xc401c980df990c4153febcf89b1b78428be1e1f5020d...,6,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9390fa8656a161442928b442300358d82bec28b0,0x9390fA8656A161442928b442300358D82bEC28b0,2.200000e+15,4.094567,1.0,DONE,2023-04-25T16:55:55.447871+00:00,ThresholdScoreCheck,True,27.21,15,NaN,NaN,NaN,NaN
2,0x4cf20243021a45a1a493358b75ef147e264aee102ad0...,0x97589cd10d5be47bef0513d22cbd9ecb9ce06ec6f9ba...,7,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x5136cdfc4d2b1a74774f5137095f82f88af5ec99,0x0035cC37599241D007D0AbA1Fb931C5FA757f7A1,4.000000e+16,74.446665,1.0,DONE,2023-04-25T17:25:19.667155+00:00,ThresholdScoreCheck,True,28.57,15,NaN,NaN,NaN,NaN
3,0x2b032f10cb65d0b9efb758931efb30571f83b076e573...,0xec0268451dd3c3ef3b527ef6478f0e89eb4741103ce7...,16,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x45b79C6bbE15363E87C948823cca846f1baDB280,3.000000e+15,5.583500,1.0,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15,NaN,NaN,NaN,NaN
4,0x0842753b06ff41ad8507a64154262ca08edd7ad6f551...,0xa9bdf73843d882ce496bae1a489c905e26339d987c08...,29,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x5041A1C1DCC760337e99B03DB60FeAF5f6f6c802,3.000000e+15,5.583500,1.0,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15,NaN,NaN,NaN,NaN


In [21]:
# remove last four columns of the dataframe
df_output = df_merge.iloc[:, :-4]
df_output.head()

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,status,last_score_timestamp,type,success,rawScore,threshold
0,0x24a5bbf100c33c84d6adf041b80650a27e0f8a9739d6...,0x64a30a4b92a54a2ad5b7cb48106104d3eb1ca743dd09...,19,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9ba961989dd6609ed091f512be947118c40f2291,0xA26d6AEB775a9528c012B748ACe66E3B0FD574cE,5.000000e+15,9.184332,1.5,DONE,2023-04-25T13:48:59.888771+00:00,ThresholdScoreCheck,True,28.57,15
1,0x3dce13bb4ed8831dfe3382d745ed593a70a5830dcc3c...,0xc401c980df990c4153febcf89b1b78428be1e1f5020d...,6,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x9390fa8656a161442928b442300358d82bec28b0,0x9390fA8656A161442928b442300358D82bEC28b0,2.200000e+15,4.094567,1.0,DONE,2023-04-25T16:55:55.447871+00:00,ThresholdScoreCheck,True,27.21,15
2,0x4cf20243021a45a1a493358b75ef147e264aee102ad0...,0x97589cd10d5be47bef0513d22cbd9ecb9ce06ec6f9ba...,7,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x5136cdfc4d2b1a74774f5137095f82f88af5ec99,0x0035cC37599241D007D0AbA1Fb931C5FA757f7A1,4.000000e+16,74.446665,1.0,DONE,2023-04-25T17:25:19.667155+00:00,ThresholdScoreCheck,True,28.57,15
3,0x2b032f10cb65d0b9efb758931efb30571f83b076e573...,0xec0268451dd3c3ef3b527ef6478f0e89eb4741103ce7...,16,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x45b79C6bbE15363E87C948823cca846f1baDB280,3.000000e+15,5.583500,1.0,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15
4,0x0842753b06ff41ad8507a64154262ca08edd7ad6f551...,0xa9bdf73843d882ce496bae1a489c905e26339d987c08...,29,0x9E669c0A6e075F14ba9d9D98C3580aD67E20ec38,0x0000000000000000000000000000000000000000,0x524cb61b52414a8413e6edec12891b11f71a711d,0x5041A1C1DCC760337e99B03DB60FeAF5f6f6c802,3.000000e+15,5.583500,1.0,DONE,2023-04-25T17:07:33.303578+00:00,ThresholdScoreCheck,True,23.56,15


In [22]:
# save the dataframe to a csv file
df_output.to_csv('./output/vote_coefficients_output.csv', index=False)

# Statistics

In [23]:
# some simple statistics on the left join
df_merge[['id','tec_tokens_flag','tea_flag']].count()

id                 257
tec_tokens_flag    108
tea_flag            44
dtype: int64

In [24]:
# count the number of unique voters
df_merge[['voter','tec_tokens_flag','tea_flag']].drop_duplicates().count()

voter              86
tec_tokens_flag    19
tea_flag            8
dtype: int64

In [25]:
# count the number of voters that have both tec and tea tokens
df_merge[(df_merge['tec_tokens_flag']==True) & (df_merge['tea_flag']==True)][['voter','tec_tokens_flag','tea_flag']].drop_duplicates().count()

voter              4
tec_tokens_flag    4
tea_flag           4
dtype: int64